<center><h1 allign="center">Artificial Neural Networks</h1></center>
<p allign>Artificial Neural Network is a machine learning framework which tries to mimic the learning patterns of natural biological neural network. It is a network of simple elements called neurons, which receive input, change their internal state through some activation function and produce an output depending on the input and the activation function. The network forms by connecting the output of certain neurons to the input of other neurons by forming a weighted graph, where the neurons are the nodes and the connection between the neurons are the weighted directed edges. The weights and activation functions can be modified by a process called learning algorithm. In our ANN mini-project, we have considered back propagation learning algorithm.
</p>

<p allign>We have considered MNIST dataset. This dataset is for predicting handwritten digits based on handwritten images of digits. Every MNIST data point has two parts: an image of a hand-written digit(x) and a corresponding label(y), which predicts the correct digit. It is split into three parts: 55,000 data points of training data(mnist.train), 10,000 points of test data(mnist.test) and 5000 points of validation data(mnist.validation). </p>
 


In [18]:
#Loading the dataset and extracting them using tensorflow
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/sathyanarayanan/data/", one_hot=True)

Extracting /home/sathyanarayanan/data/train-images-idx3-ubyte.gz
Extracting /home/sathyanarayanan/data/train-labels-idx1-ubyte.gz
Extracting /home/sathyanarayanan/data/t10k-images-idx3-ubyte.gz
Extracting /home/sathyanarayanan/data/t10k-labels-idx1-ubyte.gz


In [19]:
# importing numpy, needed for arrays and scipy needed for calculating sigmoid activation function
import numpy as np
from scipy.special import expit

<p allign>Here, we are considering a python class named NeuralNetwork. Inside this class, we perform the following opeartions.</p>
<li> We initialise the values for number of nodes in each layer- input, hidden and output layers.</li>
<li> We randomly generate the weights from input layer to hidden layer and from hidden layer to output layer.</li>
<p allign> We call the activation function (sigmoid). Activation function takes the sum of the weighted input as an argument and returns the ouput of that neural network layer.</p>

<p align> The query function is for finding the activated value of the output. We compare this output with the output from the training data, for performance measure </p>
<p align> train function performs the forward propogation, calculates the error by gradient descent approach and then performs backward propogation, thereby updating the weights of the neurons each time.</p>



In [20]:
class NeuralNetwork:
    # init is default constructor for initialising values for all layers, learning rate
    def __init__(self, n_inodes, n_hnodes, n_onodes, learn_rate):
        ## determine the number of nodes per layer
        self.i_cnt=n_inodes
        self.h_cnt=n_hnodes
        self.o_cnt=n_onodes
    
        ##specify the learning rate
        self.lr=learn_rate
        ## Assigning some random weights between -1 and 1
        self.w_i2h=np.random.rand(self.h_cnt,self.i_cnt)-0.5
        self.w_h2o=np.random.rand(self.o_cnt,self.h_cnt)-0.5
    
        ##activation function, by calling expit, which is a library function
        self.activation = lambda x: expit(x)
    
    def query(self, input_list):
        #calculate the activated output
        inputs=np.array(input_list, ndmin=2).T
    
        hidden_in=np.dot(self.w_i2h,inputs)
        hidden_out=self.activation(hidden_in)
    
        output_in=np.dot(self.w_h2o,hidden_out)
        output_out=self.activation(output_in)
    
        return output_out
    
    def train(self, input_list, target_list):
        #forward pass
        inputs=np.array(input_list,ndmin=2).T
        targets=np.array(target_list,ndmin=2).T
    
        hidden_in=np.dot(self.w_i2h,inputs)
        hidden_out=self.activation(hidden_in)
    
        output_in=np.dot(self.w_h2o,hidden_out)
        output_out=self.activation(output_in)
    
        ##backward pass (error calculation and weights updation)
        output_err=targets-output_out
    
        hidden_err=np.dot(self.w_h2o.T,output_err)
    
        self.w_h2o+=self.lr*np.dot((output_err*output_out*(1-output_out)),hidden_out.T)
        self.w_i2h+=self.lr * np.dot((hidden_err*hidden_out*(1-hidden_out)),inputs.T)
    
    

<p align>Since each image is a 28X28 pixels we have given input nodes as 784. Also, each image has a corresponding label(represented as output), a number between 0 and 9 indicating the digit of the corresponding image. We have initialised the number of hidden nodes as 250 and the learning rate to be 0.1</p>


In [21]:
n_inodes = 784
n_hnodes = 250 # increase hidden nodes.. 
n_onodes = 10

learn_rt = 0.1

nn = NeuralNetwork(n_inodes, n_hnodes, n_onodes, learn_rt)

 <p align>We are giving epoch value as 5, which means our entire dataset is run 5 times for improving accuracy. 
 For multiplication purpose, we are changing each pixel value which are 0 by 0.01 and then for each record in the dataset we are calling the train function to perform training on the dataset </p>

In [22]:
epochs = 5

for e in range(epochs):
    for record, label in zip(mnist.train.images, mnist.train.labels):
        record[record == float(0)] = 0.01 #black images is 0, we are changing it to 0.01 because that would be appropriate for multiplication
        label[label == float(0)] = 0.01
        nn.train(record, label)

<p align> Now we use the test data mnist.test, and call query function. The np.argmaxfunction returns the index of the maximum value in a vector.  So, the correct_prediction operation returns a vector of size (m x 1) of True and False values designating whether the neural network has correctly predicted the digit. This would give us the output values which we compare with the training output values. We get the performance around 97.260%, which is pretty good. </p>

In [23]:
score = []

for record, label in zip(mnist.test.images, mnist.test.labels):
    correct_label = np.argmax(label)
    inputs = record * 0.99 + 0.01

    outputs = nn.query(inputs)
    label = np.argmax(outputs)
    
    score.append(1 if label == correct_label else 0)

In [24]:
print("Performance of test set = {0:.3f}%".format(np.array(score).mean() * 100))

Performance of test set = 97.080%


<p align> Now we perform k-fold validation. We do validation in the validation set. We repeat the same process which we did with the test set, except the fact that here we loop the entire set for 5 times, because of 5-fold validation. Now we get the performance, which is almost same as the performance with the test set.</p>
<p align>Since our dataset contains 10 different output classes, we are not plotting graph to visualise the output classes. </p>

In [25]:
score_validation = []
for kfolds in range(5):
    for record, label in zip(mnist.validation.images, mnist.validation.labels):
        correct_label = np.argmax(label)
        inputs = record * 0.99 + 0.01

        outputs = nn.query(inputs)
        label = np.argmax(outputs)
    
        score_validation.append(1 if label == correct_label else 0)

In [26]:
print("Performance of validation set = {0:.3f}%".format(np.array(score_validation).mean() * 100))

Performance of validation set = 97.340%
